1. Використати клас Matcher для виділення сутностей відповідно до варіанту. Використати файл з 7-ї лабораторної роботи (не обов’язково усі висловлювання). Продемонструвати роботу.
2. Застосувати синтаксичні залежності для визначення намірів. Використати файл з 7-ї лабораторної роботи (не обов’язково усі висловлювання).

Тематика: таксі та оренда машин. Файл taxiandcars.json.
Виділити назви вулиць за допомогою класу Matcher. Виділити висловлювання користувача, що є підтвердженнями (наприклад, Yeah, that sounds good), за допомогою шаблонів.


## Виділення назв вулиць за допомогою класу Matcher

In [1]:
import spacy
from spacy.matcher import Matcher
import json

nlp = spacy.load('en_core_web_sm')

matcher = Matcher(nlp.vocab)

street_name_pattern = [
    {'IS_DIGIT': True},
    {'IS_ALPHA': True, 'OP': '+'},
    {'LOWER': {'IN': ['street', 'st', 'road', 'rd', 'avenue', 'ave', 'boulevard', 'blvd', 'place', 'pl', 'lane', 'ln']}}
]

matcher.add('STREET_NAME', [street_name_pattern])

with open('/Users/kyryl/Desktop/TextAnalysis/Lab_7/taxiandcars.json') as file:
    dialogues = json.load(file)


def extract_street_names(text):
    doc = nlp(text)
    matches = matcher(doc)
    street_names = []
    for match_id, start, end in matches:
        span = doc[start:end]
        street_names.append(span.text)
    return street_names


for dialogue in dialogues:
    for turn in dialogue['turns']:
        if turn['speaker'] == 'USER':
            street_names = extract_street_names(turn['utterance'])
            if street_names:
                print(f'Utterance: {turn["utterance"]}')
                print(f'Street Names: {street_names}')

Utterance: Can you help me book a cab?it is for two people and we would like to go to 560 Waverley Street.
Street Names: ['560 Waverley Street']
Utterance: I need to get to 3136 Contra Loma Boulevard and a shared ride should be okay.
Street Names: ['3136 Contra Loma Boulevard']
Utterance: Can you call me at cab for one person? I need to go to 211 Quarry Road #107.
Street Names: ['211 Quarry Road']
Utterance: It's for one person and I need to go to 605 Cowper Street. I'm good with a shared ride.
Street Names: ['605 Cowper Street']
Utterance: I'm going to 20700 Lake Chabot Road.
Street Names: ['20700 Lake Chabot Road']
Utterance: It will be for 1 person to go to 1512 Locust Street.
Street Names: ['1512 Locust Street']
Utterance: I need to get to 22 Belden Place.
Street Names: ['22 Belden Place']
Utterance: Actually wait, No I need it for three people going to 2100 Monument Boulevard #8.
Street Names: ['2100 Monument Boulevard']
Utterance: I'm going to 2327 Polk Street.
Street Names: ['23

# Виділення висловлювань користувача, що є підтвердженнями

In [2]:
matcher = Matcher(nlp.vocab)

confirmation_patterns = [
    [{'LOWER': 'yeah'}, {'LOWER': 'that'}, {'LOWER': 'sounds'}, {'LOWER': 'good'}],
    [{'LOWER': 'yes'}],
    [{'LOWER': 'yep'}],
    [{'LOWER': 'sure'}],
    [{'LOWER': 'correct'}]
]

matcher.add('CONFIRMATION', confirmation_patterns)


def extract_confirmations(text):
    doc = nlp(text)
    matches = matcher(doc)
    confirmations = []
    for match_id, start, end in matches:
        span = doc[start:end]
        confirmations.append(span.text)
    return confirmations


for dialogue in dialogues:
    for turn in dialogue['turns']:
        if turn['speaker'] == 'USER':
            confirmations = extract_confirmations(turn['utterance'])
            if confirmations:
                print(f'Utterance: {turn["utterance"]}')

Utterance: Yes that's correct
Utterance: Yes, please go ahead.
Utterance: Yeah, sure, that would be okay.
Utterance: Yes, that's correct.
Utterance: Yes that will be fine, can you give a estimate of how long this ride will be, and how much will it cost?
Utterance: That's correct.
Utterance: Yeah, that's correct. What'll it cost? Also, how long is the ride for?
Utterance: Yes that's correct.
Utterance: That's the thing I want. Sure. What is the length of the ride?
Utterance: Yep.
Utterance: Yep, that's it. How long is the ride? And how much will it cost?
Utterance: Yes, I want to rent this Accord.
Utterance: That's all correct. What will it cost me?
Utterance: Yes, from Sacramento, Ca on the 4th of March
Utterance: Yes please do.
Utterance: Yes that is great.
Utterance: Yes it is correct.
Utterance: Yes that works great for me. What will the price be?
Utterance: Yes, I do. What is the cost?
Utterance: Yes, that's perfect.
Utterance: Yes please! And how much is that?
Utterance: yes, I ne

## Визначення намірів з використанням синтаксичних залежностей

In [3]:
def determine_intents(text):
    doc = nlp(text)
    intents = []
    for token in doc:
        if token.dep_ in ('ROOT', 'xcomp', 'advcl') and token.pos_ in ('VERB', 'AUX'):
            intent = {'action': token.lemma_, 'object': ''}
            for child in token.children:
                if child.dep_ in ('dobj', 'attr'):
                    intent['object'] = child.text
            intents.append(intent)
    return intents

for dialogue in dialogues:
    for turn in dialogue['turns']:
        if turn['speaker'] == 'USER':
            intents = determine_intents(turn['utterance'])
            if intents:
                print(f'Utterance: {turn["utterance"]}')
                print(f'Intents: {intents}')

Utterance: Can you help me book a cab?it is for two people and we would like to go to 560 Waverley Street.
Intents: [{'action': 'help', 'object': 'me'}, {'action': 'go', 'object': ''}]
Utterance: I am ok with shared ride and is it for one people.
Intents: [{'action': 'be', 'object': ''}]
Utterance: Yes that's correct
Intents: [{'action': 'be', 'object': ''}]
Utterance: Thanks for your timely help. At present this is what i need for now.
Intents: [{'action': 'be', 'object': ''}]
Utterance: I need a cab for two people.
Intents: [{'action': 'need', 'object': 'cab'}]
Utterance: I need to get to 3136 Contra Loma Boulevard and a shared ride should be okay.
Intents: [{'action': 'need', 'object': ''}, {'action': 'get', 'object': ''}]
Utterance: Yes, please go ahead.
Intents: [{'action': 'go', 'object': ''}]
Utterance: Thank you very much for the assistance. That'll be all for now.
Intents: [{'action': 'thank', 'object': 'you'}, {'action': 'be', 'object': ''}]
Utterance: I want to go to Sons of